# Geo Coding (緯度経度取得)・最寄駅情報取得を行う

In [1]:
import pandas as pd
import numpy as np
import urllib
import xml.etree.ElementTree as ET
import re
import pandas as pd
import pyodbc
from sqlalchemy import create_engine
import time
import pickle

In [2]:
# データベースの情報
server = 'gciteam16.database.windows.net'
database = 'mynavi-database'
username = 'gciteam16'
password = 'Password0'
port =1433

In [3]:
# 接続エンジンの作成
engine_config = "?driver=ODBC+Driver+13+for+SQL+Server?charset=shift-jis"
db_settings = {
    "host": server,
    "database": database,
    "user": username,
    "password": password,
    "port":port,
    "config_query":engine_config
}
engine = create_engine('mssql+pyodbc://{user}:{password}@{host}:{port}/{database}{config_query}'.format(**db_settings))

In [4]:
"""
# データの読み込み
sql_query = "SELECT * FROM main_table"
data = pd.read_sql_query(sql_query, con = engine)
"""

'\n# \xe3\x83\x87\xe3\x83\xbc\xe3\x82\xbf\xe3\x81\xae\xe8\xaa\xad\xe3\x81\xbf\xe8\xbe\xbc\xe3\x81\xbf\nsql_query = "SELECT * FROM main_table"\ndata = pd.read_sql_query(sql_query, con = engine)\n'

In [4]:
# コネクターの作成
cnxn = pyodbc.connect('DRIVER={ODBC Driver 13 for SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
cursor = cnxn.cursor()

In [5]:
#Sample select query
cursor.execute("SELECT @@version;") 
row = cursor.fetchone() 
while row: 
    print(row[0]) 
    row = cursor.fetchone()

Microsoft SQL Azure (RTM) - 12.0.2000.8 
	Aug 29 2017 13:06:11 
	Copyright (C) 2017 Microsoft Corporation



In [2]:
# https://drive.google.com/open?id=1lN_XnVuwtxGdZRGPFxnZTVgeG6ehmC8K に上がってるファイルは一発で読める
data = pd.read_csv("../../gci_project_storage/mynavi_data.csv", encoding="SHIFT-JIS", sep=",")
data.shape

(46469, 110)

<h3>目次：</h3><br>
・raw_data_table の作成<br>
・物件の Master Table の作成<br>
・最寄駅情報の抽出<br>
・駅の Master Table の作成<br>
・物件の Geo Coding<br>
・駅の情報を main_table に追加<br>


<h3>raw_data_table の作成

In [32]:
table_name = "raw_data_table"
#data.to_sql(table_name, con=engine, if_exists="fail", index=False)

<h3>物件の Master Table の作成

In [19]:
addresses = pd.Series(data.address.unique())
df_new = pd.DataFrame(index=addresses.index, columns=["longitude","latitude"])
df_new = pd.concat([addresses, df_new], axis=1)
df_new.columns = ["address","longitude","latitude"]
df_new.shape
data_address = df_new

<h3>物件の Geo Coding

In [7]:
sql_query = "SELECT * FROM address_master"
data = pd.read_sql_query(sql_query, con = engine)

In [5]:
data_address.dtypes

address      object
longitude    object
latitude     object
dtype: object

In [14]:
address = data_address.address[0].encode("utf-8")
print(address)
serviceurl = 'http://maps.googleapis.com/maps/api/geocode/xml?'
url = serviceurl + urllib.parse.urlencode({'sensor':'false', 'address': address})
print(url)

b'\xe6\x9d\xb1\xe4\xba\xac\xe9\x83\xbd\xe6\x96\x87\xe4\xba\xac\xe5\x8c\xba\xe6\x9c\xac\xe9\x83\xb7\xef\xbc\x94\xe4\xb8\x81\xe7\x9b\xae'
http://maps.googleapis.com/maps/api/geocode/xml?sensor=false&address=%E6%9D%B1%E4%BA%AC%E9%83%BD%E6%96%87%E4%BA%AC%E5%8C%BA%E6%9C%AC%E9%83%B7%EF%BC%94%E4%B8%81%E7%9B%AE


<br><br><br>
# 0~1000
<br><br><br>

In [4]:
result_list_0_1000 = []

In [ ]:
serviceurl = 'http://maps.googleapis.com/maps/api/geocode/xml?'

for i in range(0,1000):
    
    address = data_address.address[i].encode("utf-8")

    # Python3 系：urllib.parse.urlencode()
    # Python2 系：urllib.urlencode()
    url = serviceurl + urllib.parse.urlencode({u'sensor':u'false', u'address': address})
    print ('Retrieving', url)
    # Python3 系：urllib.request.urlopen()
    uh = urllib.request.urlopen(url)
    data = uh.read()
    #print ('Retrieved',len(data),'characters')
    #print (data)
    #print ()
    tree = ET.fromstring(data)

    results = tree.findall('result')
    #lat = float( results[0].find('geometry').find('location').find('lat').text )
    #lng = float( results[0].find('geometry').find('location').find('lng').text )
    #location = results[0].find('formatted_address').text
    
    result_list_0_1000.append(data)
    time.sleep(3)
    #print ('lat',lat,'lng',lng)
    #print (location)

    #data_address.longitude[i] = lng
    #data_address.latitude[i] = lat

In [6]:
len(result_list_0_1000)

1000

In [7]:
with open('../../gci_project_storage/result_list_0_1000.pickle', mode='wb') as f:
#    pickle.dump(result_list_0_1000, f)

In [4]:
with open('../../gci_project_storage/result_list_0_1000.pickle', mode='rb') as f:
    result_list_0_1000 = pickle.load(f)

In [22]:
get_loc_json = lambda data: ET.fromstring(data).findall('result')[0].find('geometry').find('location') if len(ET.fromstring(data).findall('result'))>0 else None 
get_lat = lambda data: float(get_loc_json(data).find('lat').text) if get_loc_json(data) != None else None
get_lng = lambda data: float(get_loc_json(data).find('lng').text) if get_loc_json(data) != None else None

sum(pd.Series([get_lat(data) for data in result_list_0_1000[0:1000]]) == None)

0

<br><br><br>
# 1000~2000
<br><br><br>

In [6]:
result_list_1000_2000 = []

In [ ]:
serviceurl = 'http://maps.googleapis.com/maps/api/geocode/xml?'

for i in range(1000,2000):
    
    address = data_address.address[i].encode("utf-8")

    # Python3 系：urllib.parse.urlencode()
    # Python2 系：urllib.urlencode()
    url = serviceurl + urllib.parse.urlencode({u'sensor':u'false', u'address': address})
    print ('Retrieving', url)
    # Python3 系：urllib.request.urlopen()
    uh = urllib.request.urlopen(url)
    data = uh.read()
    #print ('Retrieved',len(data),'characters')
    #print (data)
    #print ()
    tree = ET.fromstring(data)

    results = tree.findall('result')
    #lat = float( results[0].find('geometry').find('location').find('lat').text )
    #lng = float( results[0].find('geometry').find('location').find('lng').text )
    #location = results[0].find('formatted_address').text
    
    result_list_1000_2000.append(data)
    time.sleep(3)
    #print ('lat',lat,'lng',lng)
    #print (location)

    #data_address.longitude[i] = lng
    #data_address.latitude[i] = lat

In [8]:
len(result_list_1000_2000)

1000

In [9]:
with open('../../gci_project_storage/result_list_1000_2000.pickle', mode='wb') as f:
#    pickle.dump(result_list_1000_2000, f)

In [5]:
with open('../../gci_project_storage/result_list_1000_2000.pickle', mode='rb') as f:
    result_list_1000_2000 = pickle.load(f)

In [20]:
get_loc_json = lambda data: ET.fromstring(data).findall('result')[0].find('geometry').find('location') if len(ET.fromstring(data).findall('result'))>0 else None 
get_lat = lambda data: float(get_loc_json(data).find('lat').text) if get_loc_json(data) != None else None
get_lng = lambda data: float(get_loc_json(data).find('lng').text) if get_loc_json(data) != None else None

sum(pd.Series([get_lat(data) for data in result_list_1000_2000[0:1000]]) == None)

0

# 2000~4000

In [4]:
result_list_2000_4000 = []

In [ ]:
serviceurl = 'http://maps.googleapis.com/maps/api/geocode/xml?'

for i in range(2000,4000):
    
    address = data_address.address[i].encode("utf-8")

    # Python3 系：urllib.parse.urlencode()
    # Python2 系：urllib.urlencode()
    url = serviceurl + urllib.parse.urlencode({u'sensor':u'false', u'address': address})
    print ('Retrieving', url)
    # Python3 系：urllib.request.urlopen()
    uh = urllib.request.urlopen(url)
    data = uh.read()
    #print ('Retrieved',len(data),'characters')
    #print (data)
    #print ()
    tree = ET.fromstring(data)

    results = tree.findall('result')
    #lat = float( results[0].find('geometry').find('location').find('lat').text )
    #lng = float( results[0].find('geometry').find('location').find('lng').text )
    #location = results[0].find('formatted_address').text
    
    result_list_2000_4000.append(data)
    time.sleep(5)
    #print ('lat',lat,'lng',lng)
    #print (location)

    #data_address.longitude[i] = lng
    #data_address.latitude[i] = lat

In [6]:
len(result_list_2000_4000)

2000

In [7]:
with open('../../gci_project_storage/result_list_2000_4000.pickle', mode='wb') as f:
#    pickle.dump(result_list_2000_4000, f)

In [6]:
with open('../../gci_project_storage/result_list_2000_4000.pickle', mode='rb') as f:
    result_list_2000_4000 = pickle.load(f)

In [17]:
get_loc_json = lambda data: ET.fromstring(data).findall('result')[0].find('geometry').find('location') if len(ET.fromstring(data).findall('result'))>0 else None 
get_lat = lambda data: float(get_loc_json(data).find('lat').text) if get_loc_json(data) != None else None
get_lng = lambda data: float(get_loc_json(data).find('lng').text) if get_loc_json(data) != None else None

sum(pd.Series([get_lat(data) for data in result_list_2000_4000[0:2000]]) == None)

0

# 4000~6400

In [5]:
result_list_4000_6400 = []

In [6]:
serviceurl = 'http://maps.googleapis.com/maps/api/geocode/xml?'

for i in range(4000,6400):
    
    address = data_address.address[i].encode("utf-8")

    # Python3 系：urllib.parse.urlencode()
    # Python2 系：urllib.urlencode()
    url = serviceurl + urllib.parse.urlencode({u'sensor':u'false', u'address': address})
    #print ('Retrieving', url)
    # Python3 系：urllib.request.urlopen()
    uh = urllib.request.urlopen(url)
    data = uh.read()
    #print ('Retrieved',len(data),'characters')
    #print (data)
    #print ()
    tree = ET.fromstring(data)

    results = tree.findall('result')
    #lat = float( results[0].find('geometry').find('location').find('lat').text )
    #lng = float( results[0].find('geometry').find('location').find('lng').text )
    #location = results[0].find('formatted_address').text
    
    result_list_4000_6400.append(data)
    time.sleep(6)
    #print ('lat',lat,'lng',lng)
    #print (location)

    #data_address.longitude[i] = lng
    #data_address.latitude[i] = lat

In [7]:
len(result_list_4000_6400)

2400

In [8]:
with open('../../gci_project_storage/result_list_4000_6400.pickle', mode='wb') as f:
#    pickle.dump(result_list_4000_6400, f)

In [7]:
with open('../../gci_project_storage/result_list_4000_6400.pickle', mode='rb') as f:
    result_list_4000_6400 = pickle.load(f)

In [14]:
get_loc_json = lambda data: ET.fromstring(data).findall('result')[0].find('geometry').find('location') if len(ET.fromstring(data).findall('result'))>0 else None 
get_lat = lambda data: float(get_loc_json(data).find('lat').text) if get_loc_json(data) != None else None
get_lng = lambda data: float(get_loc_json(data).find('lng').text) if get_loc_json(data) != None else None

sum(pd.Series([get_lat(data) for data in result_list_4000_6400[0:2400]]) == None)

0

# 6400~8274

In [7]:
result_list_6400_8274 = []

In [8]:
serviceurl = 'http://maps.googleapis.com/maps/api/geocode/xml?'

for i in range(6400,8274):
    
    address = data_address.address[i].encode("utf-8")

    # Python3 系：urllib.parse.urlencode()
    # Python2 系：urllib.urlencode()
    url = serviceurl + urllib.parse.urlencode({u'sensor':u'false', u'address': address})
    #print ('Retrieving', url)
    # Python3 系：urllib.request.urlopen()
    uh = urllib.request.urlopen(url)
    data = uh.read()
    #print ('Retrieved',len(data),'characters')
    #print (data)
    #print ()
    tree = ET.fromstring(data)

    results = tree.findall('result')
    #lat = float( results[0].find('geometry').find('location').find('lat').text )
    #lng = float( results[0].find('geometry').find('location').find('lng').text )
    #location = results[0].find('formatted_address').text
    
    result_list_6400_8274.append(data)
    time.sleep(6)
    #print ('lat',lat,'lng',lng)
    #print (location)

    #data_address.longitude[i] = lng
    #data_address.latitude[i] = lat

In [9]:
len(result_list_6400_8274)

1874

In [10]:
with open('../../gci_project_storage/result_list_6400_8274.pickle', mode='wb') as f:
#    pickle.dump(result_list_6400_8274, f)

In [8]:
with open('../../gci_project_storage/result_list_6400_8274.pickle', mode='rb') as f:
    result_list_6400_8274 = pickle.load(f)

In [14]:
get_loc_json = lambda data: ET.fromstring(data).findall('result')[0].find('geometry').find('location') if len(ET.fromstring(data).findall('result'))>0 else None 
get_lat = lambda data: float(get_loc_json(data).find('lat').text) if get_loc_json(data) != None else None
get_lng = lambda data: float(get_loc_json(data).find('lng').text) if get_loc_json(data) != None else None

sum(pd.Series([get_lat(data) for data in result_list_4000_6400[0:2400]]) == None)

0

# 1~8274

In [11]:
get_loc_json = lambda data: ET.fromstring(data).findall('result')[0].find('geometry').find('location') if len(ET.fromstring(data).findall('result'))>0 else None 
get_lat = lambda data: float(get_loc_json(data).find('lat').text) if get_loc_json(data) != None else None
get_lng = lambda data: float(get_loc_json(data).find('lng').text) if get_loc_json(data) != None else None

In [20]:
with open('../../gci_project_storage/result_list_0_1000.pickle', mode='rb') as f:
    result_list_0_1000 = pickle.load(f)
with open('../../gci_project_storage/result_list_1000_2000.pickle', mode='rb') as f:
    result_list_1000_2000 = pickle.load(f)
with open('../../gci_project_storage/result_list_2000_4000.pickle', mode='rb') as f:
    result_list_2000_4000 = pickle.load(f)
with open('../../gci_project_storage/result_list_4000_6400.pickle', mode='rb') as f:
    result_list_4000_6400 = pickle.load(f)
with open('../../gci_project_storage/result_list_6400_8274.pickle', mode='rb') as f:
    result_list_6400_8274 = pickle.load(f)

whole_geocoding_data = (result_list_0_1000
    +result_list_1000_2000
    +result_list_2000_4000
    +result_list_4000_6400
    +result_list_6400_8274
    )

In [ ]:
data_address.latitude  = pd.Series([get_lat(data) for data in whole_geocoding_data])
data_address.longitude = pd.Series([get_lng(data) for data in whole_geocoding_data])

In [ ]:
with open('../../gci_project_storage/data_address.pickle', mode='wb') as f:
#    pickle.dump(data_address, f)

In [10]:
data_address.head()

,address,longitude,latitude
0,東京都文京区本郷４丁目,139.755550,35.710017
1,東京都文京区本駒込４丁目,139.756541,35.731682
2,東京都文京区音羽１丁目,139.729890,35.714008
3,東京都文京区千石４丁目,139.741245,35.729040
4,東京都文京区本郷４丁目2609,139.755550,35.710017


<h3>物件の Master Table の作成

In [11]:
table_name = "address_master"
#data_address.to_sql(table_name, con=engine, if_exists="replace", index=False)

<h3>最寄駅情報の抽出

In [3]:
# 駅名と徒歩何分かを取得する関数
remove_bracket = lambda text :re.sub(u'\(.+?\)','',re.sub(u'（.+?）', '',re.sub(u'「」', '',text))) 
remove_question_mark = lambda text :re.sub(u'\?','',text)
remove_noise = lambda text:remove_bracket(remove_question_mark(text))
remove_dot = lambda text :re.sub(u'・.+?線','',text)#"・中央線東京駅"みたいなのを除く

get_station_info_0 = lambda text: (
    re.findall(u"線(\S+?駅).*?徒歩([0-9]+?)分", remove_noise(text)) 
    + re.findall(u"ライン(\S+?駅).*?徒歩([0-9]+?)分", remove_noise(text)) 
    + re.findall(u"ライナー(\S+?駅).*?徒歩([0-9]+?)分", remove_noise(text))
)
get_station_info = lambda text:{remove_dot(t[0]):int(t[1]) for t in get_station_info_0(text)} if len(get_station_info_0(text)) > 0 else {}
get_station_name = lambda text:[remove_dot(t[0]) for t in get_station_info_0(text)] if len(get_station_info_0(text)) > 0 else []
station_names = []

# 駅の総数を調べる
count_no_st = 0
for text in data.traffic:
    station_names += get_station_name(text)
    if(len(get_station_name(text))==0):
        count_no_st += 1
station_names = pd.Series(pd.Series(station_names).unique())

print("# Station : {}".format(len(station_names)))
print("# Rental Without Station : {}".format(count_no_st)) #＜-徒歩でなく、バスで移動の物件はNoneが返り値

# Station : 307
# Rental Without Station : 81


In [4]:
print(remove_bracket(data["traffic"][1758]))
print(get_station_name(data["traffic"][1758])[0])

都営三田線白山駅?徒歩10分丸ノ内線後楽園駅?徒歩15分都営大江戸線春日駅?徒歩13分
白山駅


In [5]:
data.traffic[data.traffic.apply(lambda text: u"イナー" in get_station_name(text) )]

Series([], Name: traffic, dtype: object)

In [ ]:
text = data.traffic[245]
print(text)
for st_name in get_station_name(text):
    print(st_name)
# u"[線(ライン)(ライナー)](\S+?駅).*?徒歩([0-9]+?)分"をfindall
print(remove_noise(text))
print(re.findall(u"ライナー(.+?駅).*?徒歩([0-9]+?)分",remove_noise(text))[0][0])

In [7]:
station_info = data.traffic.apply(get_station_info)

In [ ]:
station_info.head()

# 駅のGeocoding

In [17]:
result_list_station = []

In [18]:
serviceurl = 'http://maps.googleapis.com/maps/api/geocode/xml?'

for address in data_address.address:
    
    # Python3 系：urllib.parse.urlencode()
    # Python2 系：urllib.urlencode()
    url = serviceurl + urllib.parse.urlencode({u'sensor':u'false', u'address': address})
    #print ('Retrieving', url)
    # Python3 系：urllib.request.urlopen()
    uh = urllib.request.urlopen(url)
    data = uh.read()
    #print ('Retrieved',len(data),'characters')
    #print (data)
    #print ()
    tree = ET.fromstring(data)

    results = tree.findall('result')
    #lat = float( results[0].find('geometry').find('location').find('lat').text )
    #lng = float( results[0].find('geometry').find('location').find('lng').text )
    #location = results[0].find('formatted_address').text
    
    result_list_station.append(data)
    time.sleep(4)
    #print ('lat',lat,'lng',lng)
    #print (location)

    #data_address.longitude[i] = lng
    #data_address.latitude[i] = lat

KeyboardInterrupt: 

In [ ]:
len(result_list_station)

<h3>駅の Master Table の作成

In [110]:
station_master = pd.DataFrame({ 
    'station_name' : station_names,
    'longitude' : None,
    'latitude' : None
})

In [11]:
get_loc_json = lambda data: ET.fromstring(data).findall('result')[0].find('geometry').find('location') if len(ET.fromstring(data).findall('result'))>0 else None 
get_lat = lambda data: float(get_loc_json(data).find('lat').text) if get_loc_json(data) != None else None
get_lng = lambda data: float(get_loc_json(data).find('lng').text) if get_loc_json(data) != None else None

In [ ]:
station_master.latitude  = [get_lat(data) for data in result_list_station]
station_master.longitude = [get_lng(data) for data in result_list_station]

In [113]:
table_name = "station_master"
station_master.to_sql(table_name, con=engine, if_exists="replace", index=False)